In [1]:
import pandas as pd
import os
from datetime import datetime
from dispatch_bot import nearest_coordinates

In [1]:


cur_day = datetime.now().strftime('%m_%d_%Y')
columns = ['Vehicle Type', 'Pickup City', 'Pickup State', 'Pickup ZIP', 'Delivery City', 'Delivery State', 'Delivery ZIP', 'Rate', 'Payment Method', 'Distance', 'Veh_qty', 'Vehicle Info', 'Vehicle Condition', 'Trailer Type', 'Pick-Up Date', 'Order ID', 'Additional Info', 'Company Name', 'Phone', 'Email', 'Rating', 'Pickup metro area', 'Delivery metro area', 'P_lat', 'P_lon', 'D_lat', 'D_lon']
file_path = 'csv_data/csv_hourly'
file_list = os.listdir(file_path)
df = pd.concat([pd.read_csv(f"csv_data/csv_hourly/{f}") for f in file_list], ignore_index=True)
df1 = df.sort_values('Time').drop_duplicates(subset=columns, keep='first')
df2 = df.sort_values('Time').drop_duplicates(subset=columns, keep='last')
df = df1.merge(df2, on=columns, suffixes=('_first', '_last'))
df.to_csv(f'csv_data/csv_daily/{cur_day}.csv', index=False)
for f in file_list:
    path_to_file = os.path.join(file_path, f)
    os.remove(path_to_file)

In [2]:
canada = ['AB','BC','MB','NB','NL','NS','ON','PE','QC','SK','Ca']
df_l = pd.read_csv('csv_data/live.csv').drop_duplicates(subset=['Pickup City', 'Vehicle Info', 'Delivery City']).drop_duplicates(subset=['Pickup City', 'Vehicle Info', 'Company Name']).drop_duplicates(subset=['Company Name', 'Vehicle Info', 'Delivery City']).reset_index(drop=True)
df = df_l.query('`Pickup State` not in @canada and `Delivery State` not in @canada').copy()
df = df[df['Distance'] > 0]
df['RPM'] = round(df['Rate']/df['Distance'], 2)
df = df[~df['Vehicle Info'].str.contains(r'[4-9]\sunit|1[0-5]\sunit', regex=True)]
df1 = df.query('`Vehicle Type` in ["MOTORCYCLE", "ATV"] and RPM >= 0.4 and Rate >= 300').copy()
df2 = df.query('`Vehicle Type` in ["CAR", "SUV", "PICKUP"] and RPM >= 1 and Rate >= 450').copy()
truck = (34.052235, -118.243683)
df_n = await nearest_coordinates(df1, truck, n_neighbors=15)
df_n2 = await nearest_coordinates(df2, truck, n_neighbors=25)
# df_fine = await find_veh_db3(df_n, df_n, df_n2, truck, filter_by=1.65)


In [5]:
df_l = pd.read_csv('csv_data/live.csv')
df1 = df_l.query('`Vehicle Type` == "MOTO"').copy()
df1

,Vehicle Type,Pickup City,Pickup State,Pickup ZIP,Delivery City,Delivery State,Delivery ZIP,Rate,Payment Method,Distance,...,Phone,Email,Rating,Pickup metro area,Delivery metro area,P_lat,P_lon,D_lat,D_lon,Time


In [38]:
file_path = 'csv_data/csv_hourly'
file_list = os.listdir(file_path)
len(file_list)

141